In [ ]:
!git clone https://github.com/apple/ml-depth-pro
!cd ml-depth-pro && pip install -e .

In [ ]:
!source ml-depth-pro/get_pretrained_models.sh

In [ ]:
from PIL import Image
import torch
import depth_pro

depth_pro.depth_pro.DEFAULT_MONODEPTH_CONFIG_DICT.checkpoint_uri = "../checkpoints/depth_pro.pt"

model, transform = depth_pro.create_model_and_transforms(
    device=torch.device("mps")
)
model.eval()

image, _, f_px = depth_pro.load_rgb("../demo/demo.jpg")
image = transform(image)

prediction = model.infer(image, f_px=f_px)
depth = prediction["depth"] # depth in [m]
depth

In [ ]:
import PIL
from matplotlib import pyplot as plt
import numpy as np


def visualize_inverse_depth(depth: torch.Tensor) -> PIL.Image.Image:
    inverse_depth = 1 / depth
    # Visualize inverse depth instead of depth, clipped to [0.1m;250m] range for better visualization.
    max_invdepth_vizu = min(inverse_depth.max(), 1 / 0.1)
    min_invdepth_vizu = max(1 / 250, inverse_depth.min())
    inverse_depth_normalized = (inverse_depth - min_invdepth_vizu) / (
        max_invdepth_vizu - min_invdepth_vizu
    )
    cmap = plt.get_cmap("turbo")
    color_depth = (cmap(inverse_depth_normalized)[..., :3] * 255).astype(
        np.uint8
    )

    return PIL.Image.fromarray(color_depth)

In [ ]:
visual_depth = visualize_inverse_depth(depth.cpu())

plt.imshow(visual_depth)

In [ ]:
focallength_px = prediction["focallength_px"] # focal length in pixels
focallength_px